In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

In [2]:
model_name = "erc20_contracts"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name])
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="25383372:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

2025-01-24 15:26:11 [debug    ] connecting to GOLDSKY Clickhouse client... filename=client.py lineno=36 process=32387
2025-01-24 15:26:11 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=32387
2025-01-24 15:26:11 [debug    ] loaded vault: 18 items         filename=vault.py lineno=79 process=32387
2025-01-24 15:26:11 [debug    ] initialized GOLDSKY Clickhouse client. filename=client.py lineno=41 process=32387
2025-01-24 15:26:12 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=32387
2025-01-24 15:26:12 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=41 process=32387
2025-01-24 15:26:12 [info     ] prepared 1 input batches.      filename=byblock.py lineno=88 process=32387
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain',
                                                      value='base'),
                                      PartitionColumn(name='dt',
                   

In [6]:
superchainerc20 = auxiliary_templates["erc20_contracts/superchainerc20"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "contract_creation_traces": input_datasets["blockbatch/contract_creation/create_traces_v1"].as_subquery(),
    },
)

2025-01-24 15:27:01 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=32387
2025-01-24 15:27:01 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=32387 template=erc20_contracts/superchainerc20
2025-01-24 15:27:02 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=32387


In [4]:
df = ctx.client.sql(f"SELECT * FROM superchainerc20").df()
df.head()

,network,chain_id,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,tr_from_address,tx_from_address,contract_address,...,gas_used,subtraces,trace_address,error,status,tx_method_id,code_bytelength,chain,dt,token_type
0,mainnet,8453,1737556091,25383372,0xb123e0ee0f37b4d58b22be930371b88925bc2f34f045...,0x2d0ba278a49880432850c98172cb0b341bd7dcc59ae0...,170,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x0aa39e33deaa28d9382e69e64c6fbaf0bacebfee,...,2076810,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,SuperchainERC20
1,mainnet,8453,1737555787,25383220,0x4607932b07172f8120f3a11ea10adeb1b1ed60829b8c...,0x953f4a042584c7ed0b2e3887d6ddb3317bbb6015ec08...,167,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x16519c029212ad2df3c35c735c5cdde08824401a,...,2076810,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,SuperchainERC20
2,mainnet,8453,1737556483,25383568,0x567c272eca97f75e66421f13eef60010424fba33998d...,0x1659fe5c935031bb49c92b81b2202c246fa52779e5f8...,106,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x35829de024ed7d5246f30af5e88abc4438d862c3,...,2101349,0,1,,1,0xa9b29f83,13353.0,base,2025-01-22,SuperchainERC20
3,mainnet,8453,1737555839,25383246,0xe62e2e8d5aed57dd95818075f76937253b7917cc7ca5...,0xe1e84557143229f29f0c26019ce360fc746a89dcf32b...,142,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x235ff0e59a421e550a5b4950cf2df60148684846,...,2076810,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,SuperchainERC20
4,mainnet,8453,1737556337,25383495,0x7d7b303e4f9f3e36af9f4a55b5114f2d57aba279f9d5...,0x9fa03ba2b073726106d2c9aaccb93dd51afe6c455e56...,193,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x285101f404b1bd2084c604864073c9db08027747,...,2121151,0,1,,1,0xa9b29f83,13449.0,base,2025-01-22,SuperchainERC20


In [10]:
erc20_contract_creation = auxiliary_templates["erc20_contracts/erc20_contract_creation"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "contract_creation_traces": input_datasets["blockbatch/contract_creation/create_traces_v1"].as_subquery(),
        "superchainerc20": superchainerc20,
    },
)

2025-01-24 15:27:41 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=32387
2025-01-24 15:27:41 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=32387 template=erc20_contracts/erc20_contract_creation


In [12]:
erc20_contract_creation.df()

,network,chain_id,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,tr_from_address,tx_from_address,contract_address,...,subtraces,trace_address,error,status,tx_method_id,code_bytelength,chain,dt,token_standard,token_type
0,mainnet,8453,1737555787,25383220,0x4607932b07172f8120f3a11ea10adeb1b1ed60829b8c...,0x953f4a042584c7ed0b2e3887d6ddb3317bbb6015ec08...,167,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x16519c029212ad2df3c35c735c5cdde08824401a,...,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,erc20,SuperchainERC20
1,mainnet,8453,1737556091,25383372,0xb123e0ee0f37b4d58b22be930371b88925bc2f34f045...,0x2d0ba278a49880432850c98172cb0b341bd7dcc59ae0...,170,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x0aa39e33deaa28d9382e69e64c6fbaf0bacebfee,...,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,erc20,SuperchainERC20
2,mainnet,8453,1737555839,25383246,0xe62e2e8d5aed57dd95818075f76937253b7917cc7ca5...,0xe1e84557143229f29f0c26019ce360fc746a89dcf32b...,142,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x235ff0e59a421e550a5b4950cf2df60148684846,...,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,erc20,SuperchainERC20
3,mainnet,8453,1737556337,25383495,0x7d7b303e4f9f3e36af9f4a55b5114f2d57aba279f9d5...,0x9fa03ba2b073726106d2c9aaccb93dd51afe6c455e56...,193,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x285101f404b1bd2084c604864073c9db08027747,...,0,1,,1,0xa9b29f83,13449.0,base,2025-01-22,erc20,SuperchainERC20
4,mainnet,8453,1737556269,25383461,0x490b924bada1948a2a54b21ea4c0ce98574ebf436b38...,0xf0fb89e476b4cb53328e1e07d666264a8d4d3862dc66...,165,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x1c090bdb94a380a25972b138a688bc3d20af0f42,...,0,1,,1,0xa9b29f83,13449.0,base,2025-01-22,erc20,SuperchainERC20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,mainnet,8453,1737556401,25383527,0x720f5ef3535253fa941adf503dec2a955172efb55ce9...,0x9b40b3c222799717f4fb31356933fd02c21fcf9e88ad...,64,0x8909dc15e40173ff4699343b6eb8132c65e18ec6,0x15b7ec3d2051db516fb9fc04955ef41685a5b505,0xb0c28d898de5af8bd01c1e49ddf7899d8049b069,...,0,"2,0",,1,0x60806040,11636.0,base,2025-01-22,erc20,None
57,mainnet,8453,1737556253,25383453,0x06e2649116c18c6d1197c405de36b197ccc966928d22...,0xa032158a2a3f7afa9d8fa885c7698a196de0df2861f9...,90,0x6fa18c88403a0279c93e0cea6b415d0fb8b70528,0x6fa18c88403a0279c93e0cea6b415d0fb8b70528,0x21738558f3afedb7af147952244a76d8c6fce0ce,...,0,,,1,0x60806040,6578.0,base,2025-01-22,erc20,None
58,mainnet,8453,1737556539,25383596,0x865f0dd133d34fbb250b8c916982b7f08c9df6a82551...,0xe7d4523bf31b799bdff74322e01962bbbaf32143dbef...,188,0x8909dc15e40173ff4699343b6eb8132c65e18ec6,0x922ed808a6e91548e5603b82943b95cd1d3d20ec,0x77517356dfd4385cf70b5323cb5010504d133d78,...,0,"1,0",,1,0xf305d719,11636.0,base,2025-01-22,erc20,None
59,mainnet,8453,1737556519,25383586,0xc074e1572fc162ab0f3f57d34229ea898c4f38db8b2c...,0x50a6b90a7d870d52f08475209f0350d40c6c35f8bb1d...,193,0x8909dc15e40173ff4699343b6eb8132c65e18ec6,0x922ed808a6e91548e5603b82943b95cd1d3d20ec,0xe24fbd8cfcb258c560e290b3dbc123261a36022e,...,0,"1,0",,1,0xf305d719,11636.0,base,2025-01-22,erc20,None


In [11]:
df = ctx.client.sql(f"SELECT * FROM erc20_contract_creation").df()
df.head()

,network,chain_id,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,tr_from_address,tx_from_address,contract_address,...,subtraces,trace_address,error,status,tx_method_id,code_bytelength,chain,dt,token_standard,token_type
0,mainnet,8453,1737555787,25383220,0x4607932b07172f8120f3a11ea10adeb1b1ed60829b8c...,0x953f4a042584c7ed0b2e3887d6ddb3317bbb6015ec08...,167,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x16519c029212ad2df3c35c735c5cdde08824401a,...,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,erc20,SuperchainERC20
1,mainnet,8453,1737556091,25383372,0xb123e0ee0f37b4d58b22be930371b88925bc2f34f045...,0x2d0ba278a49880432850c98172cb0b341bd7dcc59ae0...,170,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x0aa39e33deaa28d9382e69e64c6fbaf0bacebfee,...,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,erc20,SuperchainERC20
2,mainnet,8453,1737555839,25383246,0xe62e2e8d5aed57dd95818075f76937253b7917cc7ca5...,0xe1e84557143229f29f0c26019ce360fc746a89dcf32b...,142,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x235ff0e59a421e550a5b4950cf2df60148684846,...,0,1,,1,0xa9b29f83,13417.0,base,2025-01-22,erc20,SuperchainERC20
3,mainnet,8453,1737556337,25383495,0x7d7b303e4f9f3e36af9f4a55b5114f2d57aba279f9d5...,0x9fa03ba2b073726106d2c9aaccb93dd51afe6c455e56...,193,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x285101f404b1bd2084c604864073c9db08027747,...,0,1,,1,0xa9b29f83,13449.0,base,2025-01-22,erc20,SuperchainERC20
4,mainnet,8453,1737556269,25383461,0x490b924bada1948a2a54b21ea4c0ce98574ebf436b38...,0xf0fb89e476b4cb53328e1e07d666264a8d4d3862dc66...,165,0x375c15db32d28cecdcab5c03ab889bf15cbd2c5e,0x002f07b0d63e8ac14f8ef6b73ccd8caf1fef074c,0x1c090bdb94a380a25972b138a688bc3d20af0f42,...,0,1,,1,0xa9b29f83,13449.0,base,2025-01-22,erc20,SuperchainERC20
